In [1]:
import sklearn

In [2]:
print(sklearn.__version__)

0.23.2


In [3]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.linear_model import LogisticRegression

# File system manangement
import os

import joblib
import requests
from time import time
import gc

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb


In [4]:
# List files available
print(os.listdir("/Users/dkoalal/Desktop/风控proj/csv/"))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [5]:
# Training data
app_train = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/application_train.csv')
app_test = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/application_test.csv')
bureau = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/bureau.csv')
bb = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/bureau_balance.csv')
prev = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/previous_application.csv')
cc = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/credit_card_balance.csv')
ins = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/installments_payments.csv')
pos = pd.read_csv('/Users/dkoalal/Desktop/风控proj/csv/POS_CASH_balance.csv')


dic={}
dic['bur'] = bureau
dic['bb'] = bb
dic['pre'] = prev
dic['cc'] = cc
dic['ins'] = ins
dic['pos'] = pos

print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_labels = app_train['TARGET']
app_train_with_label = app_train.copy()
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)
print('app_train_with_label shape: ', app_train_with_label.shape)

Training Features shape:  (307511, 121)
Testing Features shape:  (48744, 121)
app_train_with_label shape:  (307511, 122)


In [7]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder_DL(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if ((df[col].dtype == 'object') or (df[col].dtype == 'bool'))]
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

'''
def agg_numeric_DL(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns

     # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    return agg
'''

# Function to calculate correlations with the target for a dataframe
def target_corrs_DL(df):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        print(col)
        # Skip the target column
        if col != 'TARGET':
            # Calculate correlation with the target
            corr = df['TARGET'].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

def count_categorical_DL(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

def agg_categorical_DL(df, parent_var, df_name):
    """
    Aggregates the categorical features in a child dataframe
    for each observation of the parent variable.
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    parent_var : string
        The variable by which to group and aggregate the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with aggregated statistics for each observation of the parent_var
        The columns are also renamed and columns with duplicate values are removed.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[parent_var] = df[parent_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical


def agg_numeric_DL(df, parent_var, df_name):
    """
    Groups and aggregates the numeric values in a child dataframe
    by the parent variable.
    
    Parameters
    --------
        df (dataframe): 
            the child dataframe to calculate the statistics on
        parent_var (string): 
            the parent variable used for grouping and aggregating
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated by the `parent_var` for 
            all numeric columns. Each observation of the parent variable will have 
            one row in the dataframe with the parent variable as the index. 
            The columns are also renamed using the `df_name`. Columns with all duplicate
            values are removed. 
    
    """
    
    # Remove id variables other than grouping variable
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Only want the numeric variables
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg


def aggregate_client_DL(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    
    # Aggregate the numeric columns
    df_agg = agg_numeric_DL(df, parent_var = group_vars[0], df_name = df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'object'):
    
        # Count the categorical columns
        df_counts = agg_categorical_DL(df, parent_var = group_vars[0], df_name = df_names[0])

        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = agg_numeric_DL(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])

        
    # No categorical variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric_DL(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client


In [8]:
def identify_zero_importance_features_DL(train, train_labels, iterations = 2):
    """
    Identify zero importance features in a training dataset based on the 
    feature importances from a gradient boosting model. 
    
    Parameters
    --------
    train : dataframe
        Training features
        
    train_labels : np.array
        Labels for training data
        
    iterations : integer, default = 2
        Number of cross validation splits to use for determining feature importances
    """
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    # Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    
    # Fit the model multiple times to avoid overfitting
    for i in range(iterations):

        # Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    # feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
    
    # Find the features with zero importance
    # zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    zero_features = list(np.where(feature_importances==0.0)[0])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances



In [9]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

'''
class appTrainTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        # DAYS_EMPLOYED_anom
        X['DAYS_EMPLOYED_anom'] = (X['DAYS_EMPLOYED'] == 365243)
        X['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)

        # Some simple new features (percentages)
        X['DAYS_EMPLOYED_PERC'] = X['DAYS_EMPLOYED'] / X['DAYS_BIRTH']
        X['INCOME_CREDIT_PERC'] = X['AMT_INCOME_TOTAL'] / X['AMT_CREDIT']
        X['INCOME_PER_PERSON'] = X['AMT_INCOME_TOTAL'] / X['CNT_FAM_MEMBERS']
        X['ANNUITY_INCOME_PERC'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
        X['PAYMENT_RATE'] = X['AMT_ANNUITY'] / X['AMT_CREDIT']
        X.replace(np.inf, np.nan, inplace = True)
        X.replace(-np.inf, np.nan, inplace = True)

        return X

'''

class appTrainTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        # Anoms
        X['DAYS_EMPLOYED_anom'] = (X['DAYS_EMPLOYED'] == 365243)
        X['ORGANIZATION_TYPE_anom'] = (X['ORGANIZATION_TYPE'] == 'XNA')

        X['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
        X['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
        X['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
        X['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)
        X['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

        # Hand crafted features
        X['long_employment'] = (X['DAYS_EMPLOYED'] < -2000).astype(int)
        X['retirement_age'] = (X['DAYS_BIRTH'] < -14000).astype(int)

        X['annuity_income_percentage'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
        X['car_to_birth_ratio'] = X['OWN_CAR_AGE'] / X['DAYS_BIRTH']
        X['car_to_employ_ratio'] = X['OWN_CAR_AGE'] / X['DAYS_EMPLOYED']
        X['children_ratio'] = X['CNT_CHILDREN'] / X['CNT_FAM_MEMBERS']
        X['credit_to_annuity_ratio'] = X['AMT_CREDIT'] / X['AMT_ANNUITY']
        X['credit_to_goods_ratio'] = X['AMT_CREDIT'] / X['AMT_GOODS_PRICE']
        X['credit_to_income_ratio'] = X['AMT_CREDIT'] / X['AMT_INCOME_TOTAL']
        X['days_employed_percentage'] = X['DAYS_EMPLOYED'] / X['DAYS_BIRTH']
        X['income_credit_percentage'] = X['AMT_INCOME_TOTAL'] / X['AMT_CREDIT']
        X['income_per_child'] = X['AMT_INCOME_TOTAL'] / (1 + X['CNT_CHILDREN'])
        X['income_per_person'] = X['AMT_INCOME_TOTAL'] / X['CNT_FAM_MEMBERS']
        X['payment_rate'] = X['AMT_ANNUITY'] / X['AMT_CREDIT']
        X['phone_to_birth_ratio'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_BIRTH']
        X['phone_to_employ_ratio'] = X['DAYS_LAST_PHONE_CHANGE'] / X['DAYS_EMPLOYED']

        X['cnt_non_child'] = X['CNT_FAM_MEMBERS'] - X['CNT_CHILDREN']
        X['child_to_non_child_ratio'] = X['CNT_CHILDREN'] / X['cnt_non_child']
        X['income_per_non_child'] = X['AMT_INCOME_TOTAL'] / X['cnt_non_child']
        X['credit_per_person'] = X['AMT_CREDIT'] / X['CNT_FAM_MEMBERS']
        X['credit_per_child'] = X['AMT_CREDIT'] / (1 + X['CNT_CHILDREN'])
        X['credit_per_non_child'] = X['AMT_CREDIT'] / X['cnt_non_child']

        # External sources
        X['external_sources_weighted'] = X.EXT_SOURCE_1 * 2 + X.EXT_SOURCE_2 * 3 + X.EXT_SOURCE_3 * 4
        for function_name in ['min', 'max', 'sum', 'mean', 'nanmedian']:
            X['external_sources_{}'.format(function_name)] = eval('np.{}'.format(function_name))(X[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)

        X.replace(np.inf, np.nan, inplace = True)
        X.replace(-np.inf, np.nan, inplace = True)

        return X

class dropTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self, rate):
        self.rate = rate
        self.drop = []
        return None
 
    def fit(self, X, y=None):
        drop = []
        for col in X:
            if X[col].isna().sum() > self.rate * X.shape[0]:
                drop.append(col)
        self.drop = drop
        return self
 
    def transform(self, X_copy, y=None):
        X_copy.drop(self.drop, axis = 1, inplace = True)
        return X_copy



class bureauTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, bureauinfo, y=None):
        #numeric
        bureau_agg = agg_numeric_DL(bureauinfo.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', df_name = 'bureau')
        #categorial
        bureau_counts = count_categorical_DL(bureauinfo, 'SK_ID_CURR', df_name = 'bureau')
        bureau_res = bureau_agg.join(bureau_counts, how='left', on='SK_ID_CURR')

        return bureau_res




class bbTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, bureauinfo, bbinfo, y=None):
        # Counts of each type of status for each previous loan
        bureau_balance_counts = count_categorical_DL(bbinfo, 'SK_ID_BUREAU', df_name = 'bureau_balance')
        bureau_balance_agg = agg_numeric_DL(bbinfo, 'SK_ID_BUREAU', df_name = 'bb')
        # Dataframe grouped by the loan 把上两个结果merge
        bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

        # Merge to include the SK_ID_CURR
        bureau_by_loan = bureau_by_loan.merge(bureauinfo[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
        bureau_balance_by_client = agg_numeric_DL(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', df_name = 'client')

        return bureau_balance_by_client 

#把app_train join上bureau和bb的结果共得到333features

class previousTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, previnfo, y=None):
        '''
        prev, cat_cols = one_hot_encoder(previnfo, nan_as_category= True)
        # Days 365.243 values -> nan
        prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
        prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
        prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
        prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
        prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
        '''
        previous_agg = agg_numeric_DL(previnfo, 'SK_ID_CURR', 'previous')
        previous_counts = agg_categorical_DL(previnfo, 'SK_ID_CURR', 'previous')
        previous_res = previous_agg.join(previous_counts, how='left', on='SK_ID_CURR')

        return previous_res


class posTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, posinfo, y=None):
        cash_by_client = aggregate_client_DL(posinfo, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])

        return cash_by_client #165features

class ccTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, ccinfo, y=None):
        credit_by_client = aggregate_client_DL(ccinfo, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
        return credit_by_client #381features

class installmentsTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, insinfo, y=None):
        installments_by_client = aggregate_client_DL(insinfo, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])

        return installments_by_client #106个features


In [10]:
bur = bureauTransformer_DL().transform(bureau)
bb = bbTransformer_DL().transform(bureau,bb)
pre = previousTransformer_DL().transform(prev)
pos = posTransformer_DL().transform(pos)
cc = ccTransformer_DL().transform(cc)
ins = installmentsTransformer_DL().transform(ins)

In [11]:
def join_DL(X):
    X = X.join(bur, how='left', on='SK_ID_CURR')
    X = X.join(bb, how='left', on='SK_ID_CURR')
    X = X.join(pre, how='left', on='SK_ID_CURR')
    X = X.join(pos, how='left', on='SK_ID_CURR')
    X = X.join(cc, how='left', on='SK_ID_CURR')
    X = X.join(ins, how='left', on='SK_ID_CURR')
    return X

In [12]:
X = app_train.copy()
X = join_DL(X)
X = appTrainTransformer_DL().transform(X)
X = dropTransformer_DL(0.7).fit_transform(X)

numeric_columns = []
category_columns = []

for col in X:
    
    if X[col].dtype == ('object') or X[col].dtype == ('bool'):
        category_columns.append(col)
    else:
        numeric_columns.append(col)

print("Number of categorical feature:", len(category_columns))
print("Number of numerical feature:", len(numeric_columns))
print("Feature engineering result shape", X.shape)

/Users/dkoalal/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
Number of categorical feature: 18
Number of numerical feature: 866
Feature engineering result shape (307511, 884)


In [13]:
numeric_transformer_DL = Pipeline(steps=[
    ('impute_nan', Imputer(missing_values=np.nan,strategy='median')),
    ('scaler', StandardScaler())
])

category_transformer_DL = Pipeline(steps=[
    ('impute', Imputer(strategy='constant', fill_value='missing')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preprocesser_DL = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer_DL, numeric_columns),
    ('category', category_transformer_DL, category_columns)
]) 


In [14]:
class dropZeroImportanceTransformer_DL(BaseEstimator, TransformerMixin):
    def __init__(self, zero_import):
        self.zero_import = zero_import
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X_copy, y=None):
        X_copy = np.delete(X_copy,self.zero_import, axis=1)
        return X_copy

In [15]:
zero_features = [1, 12, 15, 17, 22, 24, 71, 74, 79, 81, 82, 83, 84, 86, 87, 88, 89, 92, 94, 96, 97, 98, 99, 119, 120, 123, 143, 160, 161, 164, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 184, 185, 186, 187, 188, 189, 190, 191, 192, 196, 197, 200, 201, 202, 203, 218, 224, 225, 226, 227, 229, 235, 237, 240, 241, 242, 243, 244, 245, 277, 280, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 303, 304, 305, 307, 308, 309, 310, 311, 313, 314, 315, 317, 318, 319, 320, 321, 323, 325, 326, 327, 328, 329, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 351, 352, 353, 354, 355, 357, 359, 360, 361, 362, 363, 365, 367, 368, 370, 371, 373, 375, 376, 377, 379, 381, 385, 386, 391, 392, 393, 394, 395, 397, 401, 403, 405, 406, 407, 411, 413, 418, 419, 421, 423, 429, 430, 431, 435, 442, 457, 458, 463, 464, 467, 473, 475, 479, 500, 502, 503, 508, 511, 513, 514, 519, 520, 521, 523, 551, 558, 565, 566, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 613, 614, 615, 616, 617, 618, 619, 620, 623, 627, 628, 629, 630, 631, 641, 642, 647, 650, 665, 683, 697, 700, 701, 707, 708, 712, 733, 764, 765, 767, 769, 780, 783, 788, 792, 796, 838, 839, 843, 854, 855, 867, 870, 872, 873, 874, 877, 878, 879, 881, 882, 883, 885, 888, 889, 893, 900, 901, 902, 907, 909, 910, 913, 915, 918, 920, 921, 923, 924, 925, 927, 928, 931, 932, 934, 935, 937, 940, 942, 943, 944, 945, 946, 947, 948, 949, 950, 952, 953, 955, 956, 957, 958, 959, 961, 964, 966, 967, 969, 971, 972, 974, 978, 979, 981, 982, 983, 984, 986, 987, 989, 990, 991, 992, 994, 997, 998, 1000, 1002, 1003, 1004, 1007, 1008, 1009, 1010, 1012, 1013, 1014, 1015]

In [16]:
pipeline_pre_DL = Pipeline(steps=[
    ('app', appTrainTransformer_DL()),
    ('drop', dropTransformer_DL(0.7)),
    ('preprocesser', preprocesser_DL),
    ('feature_selection',dropZeroImportanceTransformer_DL(zero_features))
])

In [18]:
app_av_train = app_train.copy()
app_av_train = join_DL(app_av_train)

app_av_test = app_test.copy()
app_av_test = join_DL(app_av_test)

app_pre_train = pipeline_pre_DL.fit_transform(app_av_train) # drop 361 features with null importance, left 655 features
app_pre_test = pipeline_pre_DL.transform(app_av_test) 

print("Train shape after feature selection",app_pre_train.shape)
print("Test shape after feature selection",app_pre_test.shape)

/Users/dkoalal/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/Users/dkoalal/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
Train shape after feature selection (307511, 655)
Test shape after feature selection (48744, 655)


In [72]:
# # Feature selection with lgbm, get 360 features with zero importance
# zero_features, feature_importances = identify_zero_importance_features_DL(app_pre_train, train_labels)
# app_pre_train = np.delete(app_pre_train,zero_features , axis=1) # 655 features
# app_pre_test = np.delete(app_pre_test,zero_features , axis=1)

# print("Final train shape (after feature selection)",app_pre_train.shape)
# print("Final test shape (after feature selection)",app_pre_test.shape)

Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.784866	valid_0's binary_logloss: 0.505889
Early stopping, best iteration is:
[135]	valid_0's auc: 0.785743	valid_0's binary_logloss: 0.522555
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.785504	valid_0's binary_logloss: 0.505891
Early stopping, best iteration is:
[135]	valid_0's auc: 0.786607	valid_0's binary_logloss: 0.522705

There are 361 features with 0.0 importance


## Train the model and predict

In [19]:
class kfoldlgb_DL(BaseEstimator, TransformerMixin):
    def __init__(self, k):
        self.k = k
        self.clf = []
        return None
 
    def fit(self, X, y=None):
        kf = KFold(n_splits=self.k)
        X_feature = np.zeros(y.shape[0])
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            gbm = lgb.LGBMClassifier(
                nthread=4,
                n_estimators=10000,
                learning_rate=0.02,
                num_leaves=34,
                colsample_bytree=0.9497036,
                subsample=0.8715623,
                max_depth=8,
                reg_alpha=0.041545473,
                reg_lambda=0.0735294,
                min_split_gain=0.0222415,
                min_child_weight=39.3259775,
                silent=-1,
                verbose=-1, )
            gbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric= 'auc', verbose= 50, early_stopping_rounds= 200)
            self.clf.append(gbm)
            X_feature[test_index] = gbm.predict_proba(X_test)[:, 1]

        feature = app_train[['SK_ID_CURR']]
        feature['feature_dl'] = X_feature
        feature.to_csv('feature_dl.csv', index = False)
        return self
 
    def predict_proba(self, X, y=None):
        for i in range(self.k):
            tmp = self.clf[i].predict_proba(X)
            if i!=0:
                result = result + tmp/self.k
            else:
                result = tmp/self.k
        return result

In [22]:
clf = kfoldlgb_DL(6)
clf.fit(app_pre_train, train_labels)

inary_logloss: 0.205866	valid_1's auc: 0.788053	valid_1's binary_logloss: 0.238541
[1250]	training's auc: 0.869822	training's binary_logloss: 0.204836	valid_1's auc: 0.788162	valid_1's binary_logloss: 0.238505
[1300]	training's auc: 0.872029	training's binary_logloss: 0.203866	valid_1's auc: 0.788308	valid_1's binary_logloss: 0.238465
[1350]	training's auc: 0.87422	training's binary_logloss: 0.202863	valid_1's auc: 0.788383	valid_1's binary_logloss: 0.238452
[1400]	training's auc: 0.876527	training's binary_logloss: 0.201844	valid_1's auc: 0.788545	valid_1's binary_logloss: 0.238396
[1450]	training's auc: 0.878723	training's binary_logloss: 0.200852	valid_1's auc: 0.788641	valid_1's binary_logloss: 0.238375
[1500]	training's auc: 0.881076	training's binary_logloss: 0.199789	valid_1's auc: 0.788776	valid_1's binary_logloss: 0.238338
[1550]	training's auc: 0.883177	training's binary_logloss: 0.198816	valid_1's auc: 0.788879	valid_1's binary_logloss: 0.23831
[1600]	training's auc: 0.88529

kfoldlgb_DL(k=6)

## For kaggle submission

In [23]:
Y = clf.predict_proba(app_pre_test)

In [24]:
log_reg_pred = Y[:, 1]
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred
submit.to_csv('baseline8.csv', index = False)

/Users/dkoalal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Save the model

In [25]:
joblib.dump(pipeline_pre_DL, './preprocess_DL.joblib')

['./preprocess_DL.joblib']

In [26]:
joblib.dump(clf, './clf_DL.joblib')

['./clf_DL.joblib']